In [30]:
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict

yelp_reviews =pd.read_table('yelp_reviews.txt', header=None)
yelp_reviews.shape

(10000, 1)

In [31]:
sup = 0.01
min_sup = int(yelp_reviews.shape[0]*sup)

In [32]:
# Create list
yelp_review_list = yelp_reviews[0].map(lambda yelp_review: yelp_review.split(' '))

In [33]:
db = defaultdict(list)

for sid, reviews in enumerate(yelp_review_list):
    for eid, review in enumerate(reviews):
        db[review].append([sid,eid])

In [34]:
# Length 1 itemset

db1 = defaultdict(pd.DataFrame)
for key, items in db.items():
    # compute support of items
    support = len(np.unique(list(zip(*items))[0]))
    if support >= min_sup:
        db1[key] = pd.concat([db1[key], pd.DataFrame(items, columns=['sid','eid' ])], axis=1).set_index('sid')

In [35]:
with open('patterns.txt', 'w') as file:
    for key, value in db1.items():
        support = len(np.unique(value.index))
        file.write(str(support)+":"+str(key)+"\n")

In [36]:
# Helper to calculate adj items

def close(data, column1, column2):
    diff = np.absolute(data[column1]-data[column2])
    cond1 = diff<2
    cond2 = diff>0
    return data[cond1 & cond2]

# Length 2 itemset

db2 = defaultdict(pd.DataFrame)
db1copy = db1.copy()

for key1, items1 in db1.items():
    for key2, items2 in db1copy.items():

        # merge based on key sid
        temp = items1.merge(items2, left_index=True, right_index=True, how='inner', suffixes=('_x', '_y'))

        # all rows smaller than minimum
        if temp.shape[0]<min_sup:
            pass

        # see if contigious
        temp = close(temp, 'eid_x', 'eid_y')

        # support check
        support = len(np.unique(temp.index))
        if support>= min_sup:
            db2[key1+";"+key2] = temp

    # remove key from copy db
    db1copy.pop(key1)


In [37]:
# divide length 2
db2split = defaultdict()
for key, items in db2.items():

    # items where first key comes first
    idx = items['eid_x']<items['eid_y']
    # case when x comes first
    support = len(np.unique(items[idx].index))
    if support >= min_sup:
        db2split[key] = items[idx]

    # case when y comes first
    support = len(np.unique(items[~idx].index))
    if support>= min_sup:
        keys = key.split(';')
        items.columns = ['eid_y', 'eid_x']
        db2split[keys[1]+';'+keys[0]] = items[~idx]

In [38]:
# Length 3 itemsets

db3 = defaultdict(pd.DataFrame)

for key1, items1 in db2split.items():
    for key2, items2 in db2split.items():
        # compare key1 first and key2 last
        key1last = key1.split(';')[1]
        key2first = key2.split(';')[0]
        key2last = key2.split(';')[1]

        # combine if keys are same
        if key1last==key2first:
            temp = items1.merge(items2, left_index=True, right_index=True, how='inner')
            # minimum support check
            support = len(np.unique(temp.index))
            if temp.shape[0] >=min_sup:
                db3[key1+" "+key2last] = temp

In [39]:
results = db1.copy()
results.update(db2split)

In [40]:
with open("patterns.txt", 'w') as file:
    # length 1
    for key, items in results.items():
        support = len(np.unique(items.index))
        file.write(str(support)+":"+';'.join(key.split())+ '\n')